In [1]:
# Load packages and Open data

%pylab
import happi

plt.rc('text', usetex=False)
mpl.rcParams['font.family'] = 'Helvetica'
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['legend.fontsize'] = 16#20
# mpl.rcParams['legend.fontname'] = 'Comic Sans MS'
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['lines.linewidth'] = 1.2

jetcmap = plt.cm.get_cmap("jet", 9) #generate a jet map with 10 values "rainbow", "jet"
jet_vals = jetcmap(np.arange(9)) #extract those values as an array 
jet_vals[0] = [1.0, 1, 1.0, 1] #change the first value 
newcmap = mpl.colors.LinearSegmentedColormap.from_list("mine", jet_vals) 

Units = happi.Units("mm", "cm^{-3}", "ns", "V/m", "T", "km/s","MeV")

# wkdir = '/Volumes/LaCie/Data_Smilei/single_shock_1d_ne2e18_v1500_Bz20_angle90_Te100_Ti200_Lx2048_dx02_ppc1024/'
wkdir = '/Volumes/LaCie/Data_Smilei/single_shock_1d_ne2e18_v1500_Bz20_angle90_Te60_Ti20_Lx2048_dx02_ppc1024/'
# wkdir = '/Volumes/LaCie/Data_Smilei/single_shock_1d_ne2e18_v1500_Bz20_angle90_Te600_Ti200_Lx2048_dx02_ppc1024/'


S = happi.Open(wkdir, reference_angular_frequency_SI=56375055300167.87)

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib
Loaded simulation '/Volumes/LaCie/Data_Smilei/single_shock_1d_ne2e18_v1500_Bz20_angle90_Te60_Ti20_Lx2048_dx02_ppc1024/'
Scanning for Scalar diagnostics
Scanning for Field diagnostics
Scanning for Probe diagnostics
Scanning for ParticleBinning diagnostics
Scanning for RadiationSpectrum diagnostics
Scanning for Performance diagnostics
Scanning for Screen diagnostics
Scanning for Tracked particle diagnostics


In [2]:
# prepare constants, units

me = 9.1e-31
mp = 1836.*me
qe = 1.6e-19
ep = 8.9e-12  # epsilon_0
c  = 3.0e8
wr = S.namelist.w_r
de = c / wr
Lx = S.namelist.L_x.real * de * 1e3      # in mm
dx = S.namelist.d_x * de * 1e3           # in mm

Te = S.namelist.T_e * 511.e3             # in eV
ne = 1.0e18                              # in cm-3
ld = 7.43e2 * Te**0.5 * ne**(-0.5) * 10. # in mm
dt = S.namelist.d_t

B0 = S.namelist.B_z * (me * wr / qe)
wc = qe * B0 / me

In [102]:
# Function for spectrum

def spect_load_1d(wkdir,time,species,xmin,xmax):
    """
    Inputs ----
    wkdir      : string, data directory
    time       : float,  [0,1] of the time_end 
    species    : int,    which species you want to plot
                         according to your own diagnostic in the Namelist
    xmin, xmax : float,  spatial selections, NOTE: in unit of de
    
    Outputs ---
    ekin       : float,  x-axis, energy(E) in MeV
    num        : float,  y-axis, E \frac{dN}{dE} \frac{1}{N}
                                 N  is the total number of particles
                                 dN is the number of particles in each energy range (dE)
                                 dE is the energy range
    simu_time  : float,  simulation time
    """
#     S = happi.Open(wkdir)                        # No need to open the data again.
    dt = S.namelist.d_t                            # get simulation timestep
    ts = S.ParticleBinning(species).getTimesteps() # get all the timesteps available for this diag.
    iteration = int(ts[-2] * time)                 # choose the time you want by a factor of [0,1]*T_end
    simu_time = iteration * dt / wr                # convert it into real unit
    pb = S.ParticleBinning(species).get(iteration) # get the diag. data from the chosen timestep
    
    xx   = pb['x'] * de * 1e3                      # get position information and convert the unit to mm
    ekin = pb['ekin'] * 0.511                      # get energy information (E) and convert the unit to MeV -- the x-axis
    data = pb['data'][0]                           # get the particle number information
    data_selected = data[(xx>xmin) & (xx<xmax),:]  # select particle number according to position
                                                   # for our case now, this doesn't make much difference.
    
    data_ekin = np.sum(data_selected,axis=0)       # dN, ?? can't remember... you might want to check what it does
    dE = ekin[1]-ekin[0]                           # dE, energy range
    total_weight = np.sum(data)                    # N, total number of the selected portion
    num = data_ekin * ekin / total_weight / dE     # E \frac{dN}{dE} \frac{1}{N} -- the y-axis
    
    return ekin, num, simu_time

# Function for 2D streak map

def prepare_streak_2D(case, num, field_name, frame):
    
    """
    case: string, Smilei case
    num: '0', instant; '1', averaged 
    field_name: string, name of the field
    frame: float, velocity of the reference frame
    """
    var = case.Field(num,field_name,units=["mm","ns","cm^{-3}"]) # find the data
    data_var = var.getData()                           # get the data out
    data_var_2D = np.vstack(data_var)                  # transform it into 2D with (t,x)
    data_var_t = var.getTimes()                        # get the t-axis
    data_var_x = var.getAxis('x')                      # get the x-axis
    
    data_var_x_2D, data_var_t_2D = np.meshgrid(data_var_x, data_var_t, indexing='xy') # make the interpolation
    
    data_var_x_2D_frame = data_var_x_2D - frame * data_var_t_2D - Lx/2.               # change the reference frame (and the label) 
                                                                                      # note the unit used
    return data_var_x_2D, data_var_x_2D_frame, data_var_t_2D, data_var_2D

def Particle_trajectory(case, species, mass, condition):
    
    """
    case: string, Smilei case
    species: string, name of the traced particle
    mass: float, mass of the traced particle, used when calcuating momentum/energy etc.
    condition: string, Smilei formatted, to select particles satisfying a certain conditions.
    frame: float, velocity of the reference frame
    """
    
#     me = 9.1e-31
#     c  = 3.0e8
#     wr = S.namelist.w_r
#     de = c / wr 
    
    tp = case.TrackParticles(species,
                             select = condition,
                             axes=['x','px','py','pz','Ex','Ey','Bz','w'],
#                              units=['mm','ns'],                     # TODO: Use normalized unit during calculation
                             sort=True,
                            )
    data = tp.getData()                                           # get data
    time_tmp = data['times']                                      # get time, in unit of wpe^{-1}
    time_ns = np.linspace(0,tp.getTimes()[-1],time_tmp.size)      # transform time array into unit of ns.
    
    tp_px = data['px']                                            # extract all kinds of data
    tp_py = data['py']
    tp_pz = data['pz']
    tp_Ex = data['Ex']
    tp_Ey = data['Ey']
    tp_Bz = data['Bz']
    tp_x  = data['x']
    tp_weight = data['w']
    
#     tp_x_frame = tp_x * de * 1e3 - frame * time_ns              # change reference frame, do it later.
    
    tp_E  = (tp_px**2 + tp_py**2 + tp_pz**2)*(me*c)**2 / 2. / mass / qe / 1e6  # energy, in Unit of MeV
    tp_E = np.where(np.isfinite(tp_E),tp_E,0.)                                 # replace the inf/nan with 0.
    
    # find the particle with the highest energy at the last timestep 
    # (end of the simulation)
    [timestep, particle_number] = np.unravel_index(tp_E[-1].argmax(), tp_E.shape)
    
    tp_x_sub = tp_x[:,particle_number]
    tp_E_sub = tp_E[:,particle_number]
    
    return tp_x, tp_E, tp_px, tp_py, tp_pz, time_ns, tp_x_sub, tp_E_sub, tp_Ex, tp_Ey, tp_Bz, tp_weight, timestep, particle_number


In [40]:
eon1t = S.TrackParticles('eon1t',
#                  select = ,
                 axes=['px','py','pz','w'],
                 sort=False,
#                  length=None,
                 units=Units,
                )

In [41]:
data = eon1t.getData()

Loading data ...
... done


In [25]:
np.shape(data[1000]['px'])

(5120,)

In [27]:
time = data['times']

In [42]:
px0 = data[time[0]]['px']
py0 = data[time[0]]['py']
pz0 = data[time[0]]['pz']
w0  = data[time[0]]['w']

In [34]:
from scipy.stats import norm

In [62]:
mean_x,std_x=norm.fit(px0)

In [72]:
plt.hist(px0, bins=30, weights=w0,density=True)
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 30)
y = norm.pdf(x, mean_x, std_x)
print(mean,std)
plt.plot(x, y)
plt.show()

0.005142861681074555 0.011544774129557466


In [80]:
me = 9.1e-31
c  = 3e8
p0 = me*c
kb = 8.6e-5

In [77]:
S.namelist.T_e*511*1000

68.13333333333334

In [78]:
std_x / 8.617333262145e-5 / 2

66.98577029782747

In [81]:
mean_y,std_y=norm.fit(py0)
plt.hist(py0, bins=30, weights=w0,density=True)
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 30)
y = norm.pdf(x, mean_y, std_y)
print(mean_y*p0/me/1e3,std_y/kb/2)
plt.plot(x, y)
plt.show()

72.81455838029285 66.63237786153657


In [82]:
mean_y,std_y=norm.fit(pz0)
plt.hist(pz0, bins=30, weights=w0,density=True)
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 30)
y = norm.pdf(x, mean_y, std_y)
print(mean_y*p0/me/1e3,std_y/kb/2)
plt.plot(x, y)
plt.show()

-151.97059287464901 66.76736025086164


In [84]:
ene = (px0**2 + py0**2 + pz0**2)
mean_y,std_y=norm.fit(ene)
plt.hist(ene, bins=30, weights=w0,density=True)
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 30)
y = norm.pdf(x, mean_y, std_y)
print(mean_y*p0/me/1e3,std_y/kb/2)
plt.plot(x, y)
plt.show()

126.9832553768513 2.0157604628185486


In [103]:
# Fig.1, Start with phase space evolution

ps = S.ParticleBinning(0,
                       units=Units, 
                       cmap=newcmap,
                       vmin=-1,vmax=3,
                       ymin=-2000, ymax=4000,
                       xmin=Lx/2,xmax=Lx,
                       data_log=True,
                      )

happi.multiPlot(ps, 
                shape=[1,1],
                figsize=[8,4],
                skipAnimation=False,
                movie='/Users/yz/Desktop/ps_200.mp4',
                fps=15,
                dpi=300,
               )

In [17]:
# Fig.2, followed by proton energy spectrum

# Get the spectrum of three different times

ekin0, num0, t_end0 = spect_load_1d(wkdir, 1.0, 1, 0.0, Lx)
ekin1, num1, t_end1 = spect_load_1d(wkdir, 0.4, 1, 0.0, Lx)
ekin2, num2, t_end2 = spect_load_1d(wkdir, 0.2, 1, 0.0, Lx)

# Plot the figure

width  = 3.2 * 2
height = width / 1.618
fig, ax = plt.subplots()
fig.subplots_adjust(left=.2, bottom=.2, right=.9, top=.9)
ax.loglog(ekin2, num2, '-g', label='{:.1f} ns'.format(t_end2*1e9), lw=2.0)
ax.loglog(ekin1, num1, '-b', label='{:.1f} ns'.format(t_end1*1e9), lw=2.0)
ax.loglog(ekin0, num0, '-r', label='{:.1f} ns'.format(t_end0*1e9), lw=2.0)
ax.tick_params(axis='both',which='both', direction='in')
ax.set_xlabel(r'$E_{p}$ (MeV)')
ax.set_ylabel(r'$E\frac{dN}{dE}\frac{1}{N}$')
ax.set_xlim([0.009, 0.2])
ax.set_xticks([0.01, 0.05, 0.10])
ax.set_xticklabels([0.01, 0.05, 0.10])
ax.set_ylim([1e-5, 1])
ax.legend(loc='best', numpoints=1, fancybox=True, fontsize=12)
ax.grid(which='both')
fig.set_size_inches(width, height)
fig.savefig('/Users/yz/Desktop/'+'spe_latest'+'.png',bbox_inches='tight',dpi=300)

In [18]:
# Fig.3, particle trajectories illustrating the acceleration mechanism

# prepare packages

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm, LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import LogLocator

# prepare to convert to center-of-mass reference frame 
n1   = 2e18     # drifting plasma, number density
n2   = 1e18     # ambient plasma, number density
v1   = 1.5e6    # drifting velocity, m/s
v2   = 0.0
vcom = (n1*v1 + n2*v2) / (n1+n2) # center-of-mass reference frame, m/s
vcom_mm_ns = vcom * 1e-6         # in unit of mm/ns

# prepare to convert to discontinuity reference frame
vcd = 0.86 # mm/ns, read from the x-t plot.

# get data for 2D streak map
Bz_x_0, Bz_x, Bz_t, Bz = prepare_streak_2D(S, 0, 'Bz', vcom_mm_ns)
ni_x_0, ni_x, ni_t, ni = prepare_streak_2D(S, 0, 'Rho_ion1+Rho_ion2', vcom_mm_ns)

Bz_x_1, Bz_x_cd,  Bz_t_cd,  Bz_cd  = prepare_streak_2D(S, 0, 'Bz', vcd)
ni_x_1, ni_x_cd,  ni_t_cd,  ni_cd  = prepare_streak_2D(S, 0, 'Rho_ion1+Rho_ion2', vcd)
Ex_x_2, Ex_x_cd2, Ex_t_cd2, Ex_cd2 = prepare_streak_2D(S, 1, 'Ex', vcd)
Ey_x_2, Ey_x_cd2, Ey_t_cd2, Ey_cd2 = prepare_streak_2D(S, 1, 'Ey', vcd)
Bz_x_2, Bz_x_cd2, Bz_t_cd2, Bz_cd2 = prepare_streak_2D(S, 1, 'Bz', vcd)

In [ ]:
# particle selection condition
# TODO: in this Smilei formatted string, you can't use the self-defined constants.
condition_p1 = "any(t>0, ((px**2+py**2+pz**2)*(9.1e-31*3.0e8)**2/2./(1836*9.1e-31)/1.6e-19/1e6 > 0.04) * ((px**2+py**2+pz**2)*(9.1e-31*3.0e8)**2/2./(1836*9.1e-31)/1.6e-19/1e6 < 0.05))"
condition_p2 = "any(t>0, ((px**2+py**2+pz**2)*(9.1e-31*3.0e8)**2/2./(1836*9.1e-31)/1.6e-19/1e6 > 0.04) * ((px**2+py**2+pz**2)*(9.1e-31*3.0e8)**2/2./(1836*9.1e-31)/1.6e-19/1e6 > 0.05))"

# get particle tracer data

ion1t_x, ion1t_E, \
ion1t_px, ion1t_py, ion1t_pz, \
ion1t_time_ns, ion1t_x_sub, ion1t_E_sub, \
ion1t_Ex, ion1t_Ey, ion1t_Bz, \
ion1t_weight, ion1t_ts, ion1t_id = Particle_trajectory(S, 'ion1t', mp, condition_p1)

ion2t_x, ion2t_E, \
ion2t_px, ion2t_py, ion2t_pz, \
ion2t_time_ns, ion2t_x_sub, ion2t_E_sub, \
ion2t_Ex, ion2t_Ey, ion2t_Bz, \
ion2t_weight, ion2t_ts, ion2t_id = Particle_trajectory(S, 'ion2t', mp, condition_p2)

In [73]:
## TOFIX: into a well written function

nt = ion1t_time_ns.size
t_end = S.namelist.t_end / wr * 1e9
time_ns = np.linspace(0.,t_end,nt)

ion2t_x_frame = zeros_like(ion2t_x)
ion2t_x_vcd   = zeros_like(ion2t_x)
# ion2t_px_vcd  = zeros_like(ion2t_px)
# ion2t_E_vcd  = zeros_like(ion2t_E)
for i in range(106):     # Note that here the loop-number should be the num. of particles.
    ion2t_x_frame[:,i] = ion2t_x[:,i]*de*1e3 - vcom_mm_ns*time_ns - Lx/2.
    ion2t_x_vcd[:,i]   = ion2t_x[:,i]*de*1e3 - vcd       *time_ns - Lx/2.
#     ion2t_px_vcd[:,i]  = ion2t_px[:,i]*me*c - vcd*1e6*mp   # in the unit of kgm/s
#     ion2t_E_vcd[:,i]   = (ion2t_px_vcd[:,i]**2 + (ion2t_py[:,i]*me*c)**2 + (ion2t_pz[:,i]*me*c)**2) / 2. / mp / qe / 1e6
    
ion1t_x_frame = zeros_like(ion1t_x)
ion1t_x_vcd   = zeros_like(ion1t_x)
ion1t_px_vcd  = zeros_like(ion1t_px)
ion1t_E_vcd   = zeros_like(ion1t_E)
for i in range(34):
    ion1t_x_frame[:,i] = ion1t_x[:,i]*de*1e3 - vcom_mm_ns*time_ns - Lx/2.
    ion1t_x_vcd[:,i]   = ion1t_x[:,i]*de*1e3 - vcd       *time_ns - Lx/2.
#     ion1t_px_vcd[:,i]  = ion1t_px[:,i]*me*c - vcd*1e6*mp   # in the unit of kgm/s
#     ion1t_E_vcd[:,i]   = (ion1t_px_vcd[:,i]**2 + (ion1t_py[:,i]*me*c)**2 + (ion1t_pz[:,i]*me*c)**2) / 2. / mp / qe / 1e6

In [47]:
plt.plot(ion2t_x_vcd[:,5],  time_ns, lw=2.0, label='{:.1f}'.format(ion2t_id))
plt.plot(ion2t_x_vcd[:,20],  time_ns, lw=2.0, label='{:.1f}'.format(ion2t_id))
plt.plot(ion2t_x_vcd[:,40],  time_ns, lw=2.0, label='{:.1f}'.format(ion2t_id))
plt.plot(ion1t_x_vcd[:,1],  time_ns, lw=2.0)
# plt.legend()

In [ ]:
plt.plot(ion2t_x_vcd[:,:],  time_ns, lw=2.0)

In [87]:
# From this point, the post-process will become quite customized.
# It will be highly dependent on what you want to show.

# Draw the figure of x-t-ni/Bz
width = 3.2 * 3
height = width / 1.618
fig, axs = plt.subplots()

# cs1 = axs.pcolormesh(Bz_x_cd2, Bz_t_cd2, np.log10(Bz_cd2**2),cmap=plt.cm.Greys)
# cs1 = axs.pcolormesh(ni_x, ni_t, ni,cmap=plt.cm.jet,vmin=0,vmax=6.0e18)
cs1 = axs.pcolormesh(ni_x_cd, ni_t_cd, ni_cd,cmap=plt.cm.jet,vmin=0,vmax=6.0e18)
cs1.cmap.set_under(color='white')
cs1.cmap.set_over(color='black')
ax1_divider = make_axes_locatable(axs)
cax1=ax1_divider.append_axes("right", size="4%", pad="2%")
# cbar1=fig.colorbar(cs1, ax=fig, cax=cax1, orientation="vertical", label=r'$log_{10}(|B_z|^2)$')
cbar1=fig.colorbar(cs1, ax=fig, cax=cax1, orientation="vertical", label=r'$n_i [cm^{-3}]$ ')
cbar1.ax.tick_params()
cax1.xaxis.set_ticks_position("top")
axs.tick_params(axis='both')
axs.set_xlabel('x [mm]')
axs.set_ylabel('t [ns]')

axs.plot(ion2t_x_vcd[:,0:100:5],  time_ns, lw=2.0)

# # axs.plot(ion2t_x_frame[:,0],  time_ns, lw=2.0, color='blue',   ls='--')
# # axs.plot(ion2t_x_frame[:,7],  time_ns, lw=2.0, color='red',  ls='--')
# # axs.plot(ion2t_x_frame[:,14], time_ns, lw=2.0, color='green', ls='--')
# # axs.plot(ion1t_x_frame[:,0],  time_ns, lw=2.0, color='black')
# # axs.plot(ion1t_x_frame[:,2],  time_ns, lw=2.0, color='m')

# # axs.plot(ion2t_x_vcd[:,0],  time_ns, lw=2.0, color='blue',   ls='--')
# axs.plot(ion2t_x_vcd[:,7],  time_ns, lw=2.0, color='red',  ls='--')
# # axs.plot(ion2t_x_vcd[:,14], time_ns, lw=2.0, color='green', ls='--')
# axs.plot(ion1t_x_vcd[:,0],  time_ns, lw=2.0, color='black')
# axs.plot(ion1t_x_vcd[:,2],  time_ns, lw=2.0, color='m')


axs.set_xlim([-2,2])
axs.set_ylim([0,3.5])

fig.set_size_inches(width,height)

fig.savefig('/Users/yz/Desktop/'+'trace_ni_xt2'+'.png',bbox_inches='tight',dpi=300)

In [74]:
# Draw the figure of x-E

width = 3.2 * 2
height = width / 1.618
fig, axs = plt.subplots()
part_id = 3
# axs.plot(ion2t_x_frame[:,0],  ion2t_E[:,0],  lw=2.0, ls='--', color='blue')
# axs.plot(ion2t_x_frame[:,7],  ion2t_E[:,7],  lw=2.0, ls='--', color='red')
# axs.plot(ion2t_x_frame[:,14], ion2t_E[:,14], lw=2.0, ls='--', color='green')
# axs.plot(ion1t_x_frame[:,0],  ion1t_E[:,0],  lw=2.0, color='black')
# axs.plot(ion1t_x_frame[:,2],  ion1t_E[:,2],  lw=2.0, color='m')

# axs.plot(ion2t_x_vcd[:,0],  ion2t_E[:,0],  lw=2.0, color='blue',   ls='--')
# axs.plot(ion2t_x_vcd[:,7],  ion2t_E[:,7],  lw=2.0, color='red',  ls='--')
# axs.plot(ion2t_x_vcd[:,14], ion2t_E[:,14], lw=2.0, color='green', ls='--')
# axs.plot(ion1t_x_vcd[:,0],  ion1t_E[:,0],  lw=2.0, color='black')
# axs.plot(ion1t_x_vcd[:,3],  ion1t_E[:,3],  lw=2.0, color='m')

# axs.plot(ion2t_x_vcd[:,0],  ion2t_E[:,0],  lw=2.0, color='blue',   ls='--')
# axs.plot(ion2t_x_vcd[:,7],  ion2t_E[:,7],  lw=2.0, color='red',  ls='--')
# axs.plot(ion2t_x_vcd[:,14], ion2t_E[:,14], lw=2.0, color='green', ls='--')
axs.plot(ion1t_x[:,part_id],  ion1t_px[:,part_id]*(me*c)/mp/1e3,  lw=2.0, color='blue', label=r'$V_x$')
axs.plot(ion1t_x[:,part_id],  ion1t_py[:,part_id]*(me*c)/mp/1e3,  lw=2.0, color='red', label=r'$V_y$')
# axs.plot(ion1t_x_vcd[:,3],  ion1t_E[:,3],  lw=2.0, color='m')

# axs.set_xlim([-2,2])
# axs.set_ylim([0,0.08])
axs.legend()
axs.set_xlabel('x [mm]')
axs.set_ylabel('v [km/s]')


fig.set_size_inches(width,height)
fig.savefig('/Users/yz/Desktop/'+'trace_x_vx_vy'+'.png',bbox_inches='tight',dpi=300)

In [73]:
# Draw the figure of x-px-py

width = 3.2 * 2
height = width 
fig, axs = plt.subplots()

# axs.plot(ion2t_px_vcd[:,0],  ion2t_py[:,0],  lw=2.0, ls='--', color='blue')
axs.plot(ion2t_px[:,7]*(me*c)/mp/1e3,  ion2t_py[:,7]*(me*c)/mp/1e3,  lw=2.0, ls='--', color='red')
# axs.plot(ion2t_px[:,14], ion2t_py[:,14], lw=2.0, ls='--', color='green')
axs.plot(ion1t_px[:,0]*(me*c)/mp/1e3,  ion1t_py[:,0]*(me*c)/mp/1e3,  lw=3.0, color='black')
axs.plot(ion1t_px[:,2]*(me*c)/mp/1e3,  ion1t_py[:,2]*(me*c)/mp/1e3,  lw=2.0, color='m')

# axs.plot(ion2t_x_vcd[:,0],  ion2t_px[:,0],  lw=2.0, color='blue',   ls='--')
# axs.plot(ion2t_x_vcd[:,7],  ion2t_px[:,7],  lw=2.0, color='red',  ls='--')
# axs.plot(ion2t_x_vcd[:,14], ion2t_px[:,14], lw=2.0, color='green', ls='--')
# axs.plot(ion1t_x_vcd[:,0],  ion1t_px[:,0],  lw=2.0, color='black')
# # axs.plot(ion1t_x_vcd[:,3],  ion1t_px[:,3],  lw=2.0, color='m')
# axs.plot(ion1t_x_vcd[:,0],  ion1t_py[:,0],  lw=2.0, color='black', ls='--')

# axs.plot(time_ns,  ion1t_px[:,0],  lw=2.0, color='black')
# # axs.plot(ion1t_x_vcd[:,3],  ion1t_px[:,3],  lw=2.0, color='m')
# axs.plot(time_ns,  ion1t_py[:,0],  lw=2.0, color='black', ls='--')

# axs.plot(time_ns,  ion2t_px[:,0],  lw=2.0, color='blue',   ls='-')
# axs.plot(time_ns,  ion2t_py[:,0],  lw=2.0, color='blue',   ls='--')

# axs.plot(ion1t_px[:,0],  ion1t_py[:,0],  lw=2.0, color='black')


axs.set_xlim([-2000,4000])
axs.set_ylim([-3000,3000])
axs.grid()

axs.set_xlabel(r'$V_x [km/s]$')
axs.set_ylabel(r'$V_y [km/s]$')

fig.set_size_inches(width,height)

fig.savefig('/Users/yz/Desktop/'+'trace_vx-vy1'+'.png',bbox_inches='tight',dpi=300)

In [ ]:
ion1t_E_vcd = np.zeros_like(ion1t_E)
ion1t_px_vcd = np.zeros_like(ion1t_px)
ion1t_py_vcd = np.zeros_like(ion1t_py)
ion1t_pz_vcd = np.zeros_like(ion1t_pz)

ion1t_px_vcd

In [66]:
# Draw the figure of x-t-ni-E

width = 3.2 * 3
height = width / 1.618 
fig, axs = plt.subplots()

cs1 = axs.pcolormesh(Bz_x_cd2, Bz_t_cd2, np.log10(Bz_cd2**2),cmap=plt.cm.Greys)
# cs1 = axs.pcolormesh(ni_x, ni_t, ni,cmap=plt.cm.Greys,vmin=0,vmax=6.0e18)
# cs1 = axs.pcolormesh(ni_x_cd, ni_t_cd, ni_cd,cmap=plt.cm.Greys,vmin=0,vmax=6.0e18)
cs1.cmap.set_under(color='white')
cs1.cmap.set_over(color='black')
ax1_divider = make_axes_locatable(axs)
cax1=ax1_divider.append_axes("right", size="4%", pad="2%")
cbar1=fig.colorbar(cs1, ax=fig, cax=cax1, orientation="vertical", label=r'$log_{10}(|B_z|^2)$')
# cbar1=fig.colorbar(cs1, ax=fig, cax=cax1, orientation="vertical", label=r'$n_i [cm^{-3}]$ ')
cbar1.ax.tick_params()
cax1.xaxis.set_ticks_position("top")
axs.tick_params(axis='both')
axs.set_xlabel('x [mm]',fontsize=18)
axs.set_ylabel('t [ns]',fontsize=18)

axs.set_xlim([-2,2])
axs.set_ylim([0,4.0])


id = 1
# points = np.array([ion2t_x_frame[:,0], time_ns]).T.reshape(-1, 1, 2)
points = np.array([ion1t_x_vcd[:,id], time_ns]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

norm = plt.Normalize(ion1t_E[:,id].min(), ion1t_E[:,id].max())
lc = LineCollection(segments, norm=norm,cmap=plt.cm.jet)
# Set the values used for colormapping
lc.set_array(ion1t_E[:,id])
lc.set_linewidth(3)
line = axs.add_collection(lc)

id = 3
points2 = np.array([ion2t_x_vcd[:,id], time_ns]).T.reshape(-1, 1, 2)
segments2 = np.concatenate([points2[:-1], points2[1:]], axis=1)

norm2 = plt.Normalize(ion2t_E[:,id].min(), ion2t_E[:,id].max())
lc2 = LineCollection(segments2, norm=norm2,cmap=plt.cm.jet)
# Set the values used for colormapping
lc2.set_array(ion2t_E[:,id])
lc2.set_linewidth(3)
line2 = axs.add_collection(lc2)

id = 25
points3 = np.array([ion2t_x_vcd[:,id], time_ns]).T.reshape(-1, 1, 2)
segments3 = np.concatenate([points3[:-1], points3[1:]], axis=1)

norm3 = plt.Normalize(ion2t_E[:,id].min(), ion2t_E[:,id].max())
lc3 = LineCollection(segments3, norm=norm3,cmap=plt.cm.jet)
# Set the values used for colormapping
lc3.set_array(ion2t_E[:,id])
lc3.set_linewidth(3)
line3 = axs.add_collection(lc3)

id = 35
points4= np.array([ion2t_x_vcd[:,id], time_ns]).T.reshape(-1, 1, 2)
segments4 = np.concatenate([points4[:-1], points4[1:]], axis=1)

norm4 = plt.Normalize(ion2t_E[:,id].min(), ion2t_E[:,id].max())
lc4 = LineCollection(segments4, norm=norm4,cmap=plt.cm.jet)
# Set the values used for colormapping
lc4.set_array(ion2t_E[:,id])
lc4.set_linewidth(3)
line4 = axs.add_collection(lc4)

cax0=fig.add_axes([0.12,0.93,0.72,0.02])
cbar0=fig.colorbar(lc, ax=fig, cax=cax0, orientation="horizontal")
cbar0.ax.tick_params()
cax0.xaxis.set_ticks_position("bottom")
cbar0.ax.set_title('E [MeV]',fontsize=18)

# fig.set_size_inches(width, height)
# fig.savefig('/Users/yz/Desktop/'+'trace_latest1'+'.png',bbox_inches='tight',dpi=300)

Text(0.5,1,'E [MeV]')

In [236]:
# Draw the figure of x-t-ni-Fx

# prepare Lorentz force
# Here, the Ex contribution is not small, but high-frequency.
# fx = qe * (ion2t_Ex[:,0]*me*wr*c/qe + ion2t_py[:,0]*me*c/mp * ion2t_Bz[:,0]*me*wr/qe)
fx = qe * (ion2t_py[:,0]*me*c/mp * ion2t_Bz[:,0]*me*wr/qe)
fy = qe * (ion2t_px[:,0]*me*c/mp * ion2t_Bz[:,0]*me*wr/qe) * -1.0

# Prepare the work done by Lorentz force
xx  = ion2t_x[:,0] * de
# ddx = zeros_like(xx)
# wx  = zeros_like(xx)
# for i in range(xx.size-1):
#     ddx[i+1] = xx[i+1] - xx[i]
#     wx[i+1]  = wx[i] + fx[i+1] * ddx[i+1]
wx = fx * xx
wx_norm = wx / wx.max()
wx_mev = wx / qe / 1e6  # convert it into MeV
# dde = zeros_like(ion2t_E[:,0])
# for i in range(dde.size-1):
#     dde[i+1] = ion2t_E[i+1,0] - ion2t_E[i,0]

width = 3.2 * 3
height = width / 1.618 / 2 
fig, axs = plt.subplots()

# cs1 = axs.pcolormesh(Bz_x, Bz_t, np.log10(Bz**2),cmap=plt.cm.Greys)
# cs1 = axs.pcolormesh(ni_x, ni_t, ni,cmap=plt.cm.Greys,vmin=0,vmax=6.0e18)
cs1 = axs.pcolormesh(ni_x_cd, ni_t_cd, ni_cd,cmap=plt.cm.Greys,vmin=0,vmax=6.0e18)
# cs1 = axs.pcolormesh(ni_x_0, ni_t_0, ni_0,cmap=plt.cm.Greys,vmin=0,vmax=6.0e18)
cs1.cmap.set_under(color='white')
cs1.cmap.set_over(color='black')
ax1_divider = make_axes_locatable(axs)
cax1=ax1_divider.append_axes("right", size="4%", pad="2%")
# cbar1=fig.colorbar(cs1, ax=fig, cax=cax1, orientation="vertical", label=r'$log_{10}(|B_z|^2)$')
cbar1=fig.colorbar(cs1, ax=fig, cax=cax1, orientation="vertical", label=r'$n_i [cm^{-3}]$ ')
cbar1.ax.tick_params()
cax1.xaxis.set_ticks_position("top")
axs.tick_params(axis='both')
axs.set_xlabel('x [mm]')
axs.set_ylabel('t [ns]')

axs.set_xlim([-2,2])
axs.set_ylim([0,3.5])

# points = np.array([ion2t_x_frame[:,0], time_ns]).T.reshape(-1, 1, 2)
points = np.array([ion2t_x_vcd[:,0], time_ns]).T.reshape(-1, 1, 2)
# points = np.array([ion2t_x[:,0]*de*1e3, time_ns]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

norm = plt.Normalize(-1.0, 1.0)
# norm = plt.Normalize(ion2t_E[:,0].min(), ion2t_E[:,0].max())
lc = LineCollection(segments, norm=norm,cmap=plt.cm.seismic)
# lc = LineCollection(segments, norm=norm,cmap=newcmap)
# Set the values used for colormapping
# lc.set_array(dde)
lc.set_array(wx_norm)
lc.set_linewidth(5)
line = axs.add_collection(lc)

cax0=fig.add_axes([0.12,0.93,0.72,0.02])
cbar0=fig.colorbar(lc, ax=fig, cax=cax0, orientation="horizontal")
cbar0.ax.tick_params()
cax0.xaxis.set_ticks_position("bottom")
cbar0.ax.set_title('Wx [A.U.]',fontsize=18)
# cbar0.ax.set_title('Fx [kgm/s2]',fontsize=18)
# fig.set_size_inches(width, height)

Text(0.5,1,'Wx [A.U.]')

In [142]:
# Draw the figure of t-E

width = 3.2 * 3
height = width / 1.618 / 2
fig, axs = plt.subplots()

axs.plot(time_ns,  ion2t_E[:,0],  lw=2.0, ls='--', color='blue', label='ID:0, from gas')
axs.plot(time_ns,  ion2t_E[:,7],  lw=2.0, ls='--', color='red', label='ID:7, from gas')
axs.plot(time_ns, ion2t_E[:,14], lw=2.0, ls='--', color='green', label='ID:14, from gas')
axs.plot(time_ns,  ion1t_E[:,0],  lw=2.0, color='black', label='ID:0, from drift plasma')
axs.plot(time_ns,  ion1t_E[:,2],  lw=2.0, color='m', label='ID:2, from drift plasma')

axs.legend()
axs.set_xlim([0,3.5])
axs.set_ylim([0,0.08])
axs.set_xlabel('time (ns)')
axs.set_ylabel('Energy (MeV)')

Text(0,0.5,'Energy (MeV)')

In [397]:
# Draw the figure of x-Bz-Ex-Vx

time_to_plot = 2.66 # in ns
time_cell = int(time_to_plot * Bz_x_cd2[:,0].size / t_end)

# kw = {'height_ratios':[13,4]}
fig, (ax0,ax1,ax2,ax3,ax4) = plt.subplots(5,1, sharex=True)


ax0.plot(Bz_x_cd2[time_cell,:],Bz_cd2[time_cell,:])
ax0.set_ylabel(r'$B_z/B_0$')
ax1.plot(Ex_x_cd2[time_cell,:],Ex_cd2[time_cell,:])
ax1.set_ylabel(r'$E_x/E_0$')
ax2.plot(Ey_x_cd2[time_cell,:],Ey_cd2[time_cell,:])
ax2.set_ylabel(r'$E_y/E_0$')
ax3.plot(ni_x_cd[time_cell,:],ni_cd[time_cell,:]/1e18)
ax3.set_ylabel(r'$n_i/n_0$')

# find the same timestep for phase-space data
ts = S.ParticleBinning(0).getTimesteps()
# it = int(ts[-1]*time_to_plot/t_end)    # here, not exactly the timestep, try to fix.
it = 1499850

# get the acorresponding data out
ps = S.ParticleBinning(0).get(it)
xx = ps['x']*de*1e3 - time_to_plot*vcd - Lx/2.   # convert unit to mm
vx = ps['vx']*c*1e-3  # convert unit to km/s
da = np.log10(ps['data'][0])

xmin = 6.4 - time_to_plot*vcd - Lx/2.
xmax = 7.0 - time_to_plot*vcd - Lx/2.
xcmi = int((xmin-xx.min())*xx.size/(xx.max()-xx.min()))
xcma = int((xmax-xx.max())*xx.size/(xx.max()-xx.min()))

vmin = -200.0
vmax = 2200.0
vcmi = int((vmin-vx.min())*vx.size/(vx.max()-vx.min()))
vcma = int((vmax-vx.min())*vx.size/(vx.max()-vx.min()))
ax4.imshow(da[xcmi:xcma,vcmi:vcma].T,
          extent=[xmin,xmax,vmin,vmax],
          origin='lower',
          aspect='auto',
          cmap=newcmap)
ax4.set_ylabel(r'$v_x$ [km/s]')

ax4.set_xlabel('x [mm]')
fig.suptitle('time = {:.2f} ns'.format(time_to_plot))


/usr/local/Tools/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in log10


Text(0.5,0.98,'time = 2.66 ns')

In [410]:
xmi = 6.4
xma = 7.2
vmi = -200.0
vma = 2000.0

Bz0 = S.Field(1,'Bz',units=['ns','mm'],xmin=xmi,xmax=xma)
Bz0.set(xlabel='')
Ex0 = S.Field(1,'Ex',units=['ns','mm'],xmin=xmi,xmax=xma)
Ex0.set(xlabel='',title='')
ni0 = S.Field(0,'Rho_ion1+Rho_ion2',units=['ns','mm'],xmin=xmi,xmax=xma)
ni0.set(xlabel='',title='',ylabel='Ni(N_0)')
xpx = S.ParticleBinning(0,units=['ns','mm','km/s'],cmap=newcmap,
                        xmin=xmi,xmax=xma, 
                        ymin=vmi,ymax=vma,
                        cbaspect=0,                
                        data_log=True)
xpx.set(title='')

happi.multiPlot(Bz0, Ex0, ni0, xpx, skipAnimation=True, shape=[4,1],timesteps=1500000)

In [413]:
plt.pcolormesh(ni_x_1, ni_t_cd, ni_cd,cmap=newcmap,vmin=0,vmax=6.0e18)
plt.colorbar()
plt.xlabel('x [mm]')
plt.ylabel('t [ns]')

Text(0,0.5,'t [ns]')

In [394]:
time_to_plot*vcd

2.2876000000000003

In [310]:
plt.plot(Ex_x_cd2[228,:],Ex_cd2[228,:])
plt.plot(Bz_x_cd2[228,:],Bz_cd2[228,:])

plt.title('time = {:.2f} ns'.format(3.5*228/300))

Text(0.5,1,'time = 2.66 ns')

In [311]:
b = S.ParticleBinning(0)

In [386]:
ts = b.getTimesteps()
it = 1499850
st = it * dt / wr
st

2.6604851951170216e-09

In [320]:
ps = b.get(it)

In [359]:
xx = ps['x']*de*1e3
vx = ps['vx']*c*1e-3
da = np.log10(ps['data'][0])

/usr/local/Tools/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log10
  This is separate from the ipykernel package so we can avoid doing imports until


In [380]:
ts = b.getTimesteps()

In [384]:
it = int(ts[-1]*time_to_plot/t_end)


1499426.513337367

In [75]:
Lx

10.898437202919613

In [76]:
t_end

3.5476683603253947

In [ ]:
import matplotlib.font_manager
from IPython.core.display import HTML

def make_html(fontname):
    return "<p>{font}: <span style='font-family:{font}; font-size: 24px;'>{font}</p>".format(font=fontname)

code = "\n".join([make_html(font) for font in sorted(set([f.name for f in matplotlib.font_manager.fontManager.ttflist]))])

HTML("<div style='column-count: 2;'>{}</div>".format(code))
